In [ ]:
pip install d2l==0.17.6

In [ ]:
pip uninstall matplotlib


In [ ]:
pip install --upgrade pip

In [ ]:

pip install matplotlib


2.4.1 导数绘图

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib_inline import backend_inline
from d2l import torch as d2l

def f(x):
  return 3*x**2-4*x

def numerical_lim(f,x,h):
  return (f(x+h)-f(x))/h

h=0.1
for i in range(5):
  print(f'h={h:.5f}, numerical limit={numerical_lim(f,1,h):.5f}')
  h*=0.1

def use_svg_display():
  backend_inline.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)):
  use_svg_display()
  d2l.plt.rcParams['figure.figsize'] = figsize

def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
  axes.set_xlabel(xlabel)
  axes.set_ylabel(ylabel)
  axes.set_xlim(xlim)
  axes.set_ylim(ylim)
  axes.set_xscale(xscale)
  axes.set_yscale(yscale)
  if legend:
    axes.legend(legend)
  axes.grid()

def plot(X, Y=None, xlabel=None, ylabel=None, legend=None, xlim=None, ylim=None, xscale='linear', yscale='linear', fmts=('-', 'm--', 'g-.', 'r:'), figsize=(3.5, 2.5), axes=None):
  if legend is None:
    legend=[]

  set_figsize(figsize)
  axes = axes if axes else d2l.plt.gca()

  def has_one_axis(X):
    return (hasattr(X, "ndim") and X.ndim == 1 or isinstance(X, list) and not hasattr(X[0], "__len__"))

  if has_one_axis(X):
    X=[X]
  if Y is None:
    X, Y = [[]] * len(X), X
  elif has_one_axis(Y):
    Y = [Y]
  if len(X) != len(Y):
    X = X*len(Y)
  axes.cla()
  for x, y, fmt in zip(X, Y, fmts):
    if len(x):
      axes.plot(x, y, fmt)
    else:
      axes.plot(y, fmt)
  set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend)

x = np.arange(0, 3, 0.1)
plot(x, [f(x), 2*x-3], 'x', 'f(x)', legend=['f(x)', 'Tangent line (x=1)'])


2.6.1 概率

In [ ]:
%matplotlib inline
import torch
from torch.distributions import multinomial
from d2l import torch as d2l

fair_probs = torch.ones([6])/6
# multinomial.Multinomial(1, fair_probs).sample()
# multinomial.Multinomial(10, fair_probs).sample()

# count = multinomial.Multinomial(1000, fair_probs).sample()
# count/1000

counts = multinomial.Multinomial(10, fair_probs).sample((500,))
cum_counts = counts.cumsum(dim=0)
estimates = cum_counts/cum_counts.sum(dim=1, keepdims=True)

d2l.set_figsize((6,4.5))
for i in range(6):
  d2l.plt.plot(estimates[:,i].numpy(), label=("P(die=" + str(i+1) + ")"))
d2l.plt.axhline(y=0.167, color='black', linestyle='dashed')
d2l.plt.gca().set_xlabel('Groups of experiments')
d2l.plt.gca().set_ylabel('Estimated probability')
d2l.plt.legend()


解析解：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# 100行1列的x1
X = 2 * np.random.rand(100, 1)
#这个公式就等于： 真实的y=预测的y+误差；预测的y=x0*a+b*x，因为x0恒为1，所以就是a+b*x
y = 4 + 3 * X + np.random.randn(100, 1)
# 整合x0和x1 x0是100行1
X_b = np.c_[np.ones((100, 1)), X]

#用解析解公式求theta
# linalg是线形和代数这两个英文词组合起来的，inv是在求逆，dot是点乘，T是求转置
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print(theta_best)

# 但是在机器学习中数据为王的思想，数据特别多，不适合上面那种方法，太慢了，
# 预测
X_new = np.array([[0], [2]])# 2行一列的数组x1，值为0，2
X_b_new = np.c_[(np.ones((2, 1))), X_new] #x0，x1整合
print(X_b_new)
# 根据上面求得的模型theta预测未来
y_predict = X_b_new.dot(theta_best)
#将0，和2分别带入预测的y=4+3*x，看得到的值是否和y_predict差不多，肯定会有误差的，接近就行
print(y_predict)

plt.plot(X_new, y_predict, 'r-')
plt.plot(X, y, 'b.')
plt.axis([0, 2, 0, 15])
plt.show()

3.1.2 向量化加速

In [ ]:
%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

n = 10000
a = torch.ones(n)
b = torch.ones(n)

# 定义一个计时器
class Timer:
  def __init__(self):
    self.times = []
    self.start()
    pass

  def start(self):
    #启动计时器
    self.tik = time.time()

  def stop(self):
    #停止计时器并将时间记录在列表中
    self.times.append(time.time() - self.tik)
    return self.times[-1]

  def avg(self):
    #返回平均时间
    return sum(self.times)/len(self.times)

  def sum(self):
    #返回时间总和
    return sum(self.times)

  def cumsum(self):
    #返回累计时间
    return np.array(self.times).cumsum().tolist()

c = torch.zeros(n)
timer = Timer()
for i in range(n):
  c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

timer.start()
d = a + b
f'{timer.stop():.5f} sec'

3.2 线性回归

In [ ]:
from numpy.linalg import LinAlgError
%matplotlib inline
import torch
import random
from d2l import torch as d2l

#生成数据集
def synthetic_data(w, b, num_examples):
  '''生成y=Xw+b+噪声'''
  X = torch.normal(mean=0, std=1, size=(num_examples, len(w)))
  y = torch.matmul(X, w) + b
  y += torch.normal(mean=0, std=0.01, size=y.shape)
  return X, y.reshape(-1,1)

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print('features:', features[0], '\nlables:', labels[0])

d2l.set_figsize()
d2l.plt.scatter(features[:, 1].detach().numpy(), labels.detach().numpy(), 1);

#读取数据集
def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  # 这些样本是随机读取的，没有特定的顺序
  random.shuffle(indices) # 随机打乱
  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
    yield features[batch_indices], labels[batch_indices] # 生成器，在for循环中迭代

batch_size = 10
for X, y in data_iter(batch_size, features, labels):
  print(X, '\n', y)
  break

# 初始化模型参数
w = torch.normal(mean=0, std=0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

#定义模型
def linreg(X, w, b):
  '''线性回归模型'''
  return torch.matmul(X, w) + b

# 定义损失函数
def squared_loss(y_hat, y):
  return (y_hat-y.reshape(y_hat.shape)) ** 2 / 2

# 定义优化方法
def sgd(params, lr, batch_size):
  '''小批量随机梯度下降'''
  with torch.no_grad():
    for param in params:
      param -= lr * param.grad/batch_size
      param.grad.zero_()

# 训练
lr = 0.03 # 学习率
num_epochs = 3 # 轮数
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
  for X, y in data_iter(batch_size, features, labels):
    l = loss(net(X, w, b), y) # X和y的小批量损失
    l.sum().backward()
    sgd([w,b], lr, batch_size) # 使用参数的梯度更新参数
  with torch.no_grad():
    train_l = loss(net(features, w, b), labels)
    print(f'epoch{epoch+1}, loss {float(train_l.mean()):f}')

print(f'w的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - b}')

3.3 线性回归简洁实现

In [ ]:
import numpy as np
import torch
from torch.utils import data
from torch import nn
from d2l import torch as d2l

# 生成数据集
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 读取数据集
def load_array(data_arrays, batch_size, is_train=True):
  '''构造一个Pytorch迭代器'''
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

# 定义模型
net = nn.Sequential(nn.Linear(2,1)) # 2表示输入特征形状，1表示输出特征形状

# 初始化模型参数
net[0].weight.data.normal_(mean=0,std=0.01)
net[0].bias.data.fill_(0)

# 定义损失函数
loss = nn.MSELoss() # 平方L2范数

# 定义优化方法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X), y)
    trainer.zero_grad()
    l.backward()
    trainer.step()
  l = loss(net(features), labels)
  print(f'epoch {epoch+1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)